# Machine Learning for features

## Import packages

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import shap
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from tqdm import tqdm

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Load data

In [2]:
# load data
df_all = pd.read_csv('../data/paras.txt', sep='\t')

# 切分数据
df_ASD_AC = df_all[df_all['Disease'].isin(['ASD', 'ASD_Cancer'])]
df_Cancer_AC = df_all[df_all['Disease'].isin(['Cancer', 'ASD_Cancer'])]

## RFE

### Cancer vs ASD_Cancer

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Features and labels
X = df_Cancer_AC[['Co.evolution','Consurf_Score','Entropy','RASA','ddG',
             'Betweenness','Closeness','Degree','Eigenvector',
             'Clustering.coefficient','Effectiveness','Sensitivity',
             'MSF','DFI','Stiffness']]
y = df_Cancer_AC['Disease']
X1 = X
y1 = y
X1 = X1.reset_index(drop=True)
y1 = y1.reset_index(drop=True)
shuffle_index = np.random.permutation(X1.index)
X1 = X1.iloc[shuffle_index]
y1 = y1.iloc[shuffle_index]
y1_encode = y1.map({'ASD_Cancer': 1, 'Cancer': 0})
select_feature = []

# RFE
for i in range(15):
    model = LGBMClassifier(verbose = -1, n_estimators = 1000, max_depth = 5) 
    rfe = RFE(model, n_features_to_select= i+1)
    rfe.fit(X1, y1_encode)
    selected_features =list(X.columns[rfe.support_])
    if 'Degree' in selected_features:
        selected_features.remove('Degree')
    if "Effectiveness" not in selected_features \
        and "Sensitivity" not in selected_features \
        and "Stiffness" not in selected_features \
        and "MSF" not in selected_features \
        and "DFI" not in  selected_features:
            selected_features.append("Effectiveness")
    if "ddG" not in selected_features:
            selected_features.append("ddG")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X1[selected_features], y1_encode, cv=cv)
    print("----------------"+ str(i+1) + "---------------")
    # selected_features = X1.columns[rfe.support_]
    print("selected features:", selected_features)
    print("cross validation score:", scores.mean())
    select_feature.append(selected_features)
    select_feature.append(scores.mean())

----------------1---------------
selected features: ['Closeness', 'Effectiveness', 'ddG']
cross validation score: 0.9480130902290791
----------------2---------------
selected features: ['Closeness', 'Effectiveness', 'ddG']
cross validation score: 0.9480130902290791
----------------3---------------
selected features: ['Closeness', 'Effectiveness', 'ddG']
cross validation score: 0.9480130902290791
----------------4---------------
selected features: ['Closeness', 'Eigenvector', 'Effectiveness', 'ddG']
cross validation score: 0.9544647031323048
----------------5---------------
selected features: ['Betweenness', 'Closeness', 'Eigenvector', 'Effectiveness', 'ddG']
cross validation score: 0.9588125292192615
----------------6---------------
selected features: ['Betweenness', 'Closeness', 'Eigenvector', 'Effectiveness', 'Stiffness', 'ddG']
cross validation score: 0.9544647031323048
----------------7---------------
selected features: ['Betweenness', 'Closeness', 'Eigenvector', 'Effectiveness', '

### The indicators for Cancer vs ASD_Cancer is 
['Co.evolution', 'Consurf_Score', 'RASA', 'Betweenness', 'Closeness', 'Eigenvector', 'Effectiveness', 'Sensitivity', 'Stiffness', 'ddG']

### ASD vs ASD_Cancer

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Features and labels
X = df_ASD_AC[['Co.evolution','Consurf_Score','Entropy','RASA','ddG',
             'Betweenness','Closeness','Degree','Eigenvector',
             'Clustering.coefficient','Effectiveness','Sensitivity',
             'MSF','DFI','Stiffness']]
y = df_ASD_AC['Disease']
X1 = X
y1 = y
X1 = X1.reset_index(drop=True)
y1 = y1.reset_index(drop=True)
shuffle_index = np.random.permutation(X1.index)
X1 = X1.iloc[shuffle_index]
y1 = y1.iloc[shuffle_index]
y1_encode = y1.map({'ASD_Cancer': 1, 'ASD': 0})
select_feature = []

# RFE
for i in range(15):
    model = LGBMClassifier(verbose = -1, n_estimators = 1000, max_depth = 5) 
    rfe = RFE(model, n_features_to_select= i+1)
    rfe.fit(X1, y1_encode)
    selected_features =list(X.columns[rfe.support_])
    if 'Degree' in selected_features:
        selected_features.remove('Degree')
    if "Effectiveness" not in selected_features \
        and "Sensitivity" not in selected_features \
        and "Stiffness" not in selected_features \
        and "MSF" not in selected_features \
        and "DFI" not in  selected_features:
            selected_features.append("Effectiveness")
    if "ddG" not in selected_features:
            selected_features.append("ddG")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X1[selected_features], y1_encode, cv=cv)
    print("----------------"+ str(i+1) + "---------------")
    # selected_features = X1.columns[rfe.support_]
    print("selected features:", selected_features)
    print("cross validation score:", scores.mean())
    select_feature.append(selected_features)
    select_feature.append(scores.mean())

----------------1---------------
selected features: ['Closeness', 'Effectiveness', 'ddG']
cross validation score: 0.7125356125356126
----------------2---------------
selected features: ['Closeness', 'Eigenvector', 'Effectiveness', 'ddG']
cross validation score: 0.7264957264957266
----------------3---------------
selected features: ['Betweenness', 'Closeness', 'Eigenvector', 'Effectiveness', 'ddG']
cross validation score: 0.7649572649572649
----------------4---------------
selected features: ['ddG', 'Betweenness', 'Closeness', 'Eigenvector', 'Effectiveness']
cross validation score: 0.7649572649572649
----------------5---------------
selected features: ['RASA', 'ddG', 'Betweenness', 'Closeness', 'Eigenvector', 'Effectiveness']
cross validation score: 0.7948717948717949
----------------6---------------
selected features: ['RASA', 'ddG', 'Betweenness', 'Closeness', 'Eigenvector', 'Sensitivity']
cross validation score: 0.8256410256410256
----------------7---------------
selected features: [

In [ ]:
### The indicators for ASD vs ASD_Cancer is 
['ddG', 'Betweenness', 'Closeness', 'Eigenvector', 'Sensitivity', 'RASA']

## Explaination

### Cancer vs ASD_Cancer

In [9]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# The indicators are
X = df_Cancer_AC[['Co.evolution', 'Consurf_Score', 'RASA', 'Betweenness', 
                      'Closeness', 'Eigenvector', 'Effectiveness', 'Sensitivity', 'Stiffness', 'ddG']]
y = df_Cancer_AC['Disease']
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
shuffle_index = np.random.permutation(X.index)
X = X.iloc[shuffle_index]
y = y.iloc[shuffle_index]
y = y.map({'ASD_Cancer': 1, 'Cancer': 0})
# Using LightGBM Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
d_train = lgb.Dataset(X_train, label=y_train)
d_test = lgb.Dataset(X_test, label=y_test)

# parameters
params = {
    "max_bin": 512,
    "learning_rate": 0.05,
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",
    "num_leaves": 10,
    "verbose": -1,
    "boost_from_average": True,
    "early_stopping_rounds": 50,
    "verbose_eval": 1000
}
# train the model
model = lgb.train(
    params,
    d_train,
    1000,
    valid_sets=[d_test],
)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# Save force plot
p = shap.force_plot(explainer.expected_value[1], 
                    shap_values[1], 
                    X,
                    link="logit")
shap.save_html(f'../figure/force_plot_Cancer_ASD_Cancer.html', p)

# Save summary plot
fig = shap.summary_plot(shap_values[1], X, alpha=0.75, show=False)
plt.title("Cancer vs ASD_Cancer", fontweight='bold', fontsize=15)
plt.xlabel("Impact on model output")
plt.savefig(f"../figure/summary_plot_Cancer_ASD_Cancer.pdf", bbox_inches='tight')
plt.close()

### ASD vs ASD_Cancer

In [10]:
# The indicators are
X = df_ASD_AC[['ddG', 'Betweenness', 'Closeness', 'Eigenvector', 'Sensitivity', 'RASA']]
y = df_ASD_AC['Disease']
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
shuffle_index = np.random.permutation(X.index)
X = X.iloc[shuffle_index]
y = y.iloc[shuffle_index]
y = y.map({'ASD_Cancer': 1, 'ASD': 0})
# Using LightGBM Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
d_train = lgb.Dataset(X_train, label=y_train)
d_test = lgb.Dataset(X_test, label=y_test)

# parameters
params = {
    "max_bin": 512,
    "learning_rate": 0.05,
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",
    "num_leaves": 10,
    "verbose": -1,
    "boost_from_average": True,
    "early_stopping_rounds": 50,
    "verbose_eval": 1000
}
# train the model
model = lgb.train(
    params,
    d_train,
    1000,
    valid_sets=[d_test],
)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# Save force plot
p = shap.force_plot(explainer.expected_value[1], 
                    shap_values[1], 
                    X,
                    link="logit")
shap.save_html(f'../figure/force_plot_ASD_ASD_Cancer.html', p)

# Save summary plot
fig = shap.summary_plot(shap_values[1], X, alpha=0.75, show=False)
plt.title("ASD vs ASD_Cancer", fontweight='bold', fontsize=15)
plt.xlabel("Impact on model output")
plt.savefig(f"../figure/summary_plot_ASD_ASD_Cancer.pdf", bbox_inches='tight')
plt.close()